In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import pairwise
from IPython.display import HTML
import extract_feature_vector
from IPython.display import Image as iImage
import base64
from PIL import Image
from io import BytesIO
import topfivebreeds
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import math
from bokeh.plotting import figure, show
from bokeh.io import output_notebook


ModuleNotFoundError: No module named 'extract_feature_vector'

In [2]:
pd.set_option('display.max_colwidth', -1)


In [3]:
def maketop10df(top10list):
    simpetsdf = pd.DataFrame(columns=['Picture','Pet Name','Breeds','Shelter/Location/Contact','Similarity\\nScore'])
    for simpic in top10list.index:    
        petid,picnum = simpic.split('_')
        filestr = pfdf.loc[petid,'media'][int(picnum)-1][0]
        imgstr = '''<img style="border:3px solid white" src="'''+filestr+'''">'''
        simpetsdf.loc[simpic,'Picture'] = imgstr
        simpetsdf.loc[simpic,'Pet Name'] = pfdf.loc[petid,'petname']
        if pfdf.loc[petid,'breed2']:
            simpetsdf.loc[simpic,'Breeds'] = ',\\n'.join([pfdf.loc[petid,'breed1'],pfdf.loc[petid,'breed2']])
        else:
            simpetsdf.loc[simpic,'Breeds'] = pfdf.loc[petid,'breed1']
        infodict={}
        for info in ['email','city','state','zip', 'country', 'sheltername' ]:
            if pfdf.notnull().loc[petid,info]:
                infodict[info]=pfdf.loc[petid,info]  
            else:
                infodict[info] = ''
        simpetsdf.loc[simpic,'Shelter/\\nLocation/\\nContact'] = str(infodict['sheltername'] + '\\n'
                                                       + infodict['city']+', '
                                                       + infodict['state'] + ' ' 
                                                       + infodict['zip'] + ' '  
                                                       + infodict['country'] + '\\n'
                                                       + infodict['email'])

        simpetsdf.loc[simpic,'Similarity\\nScore'] = "{0:.4f}".format(top10list.loc[simpic])
    return simpetsdf

In [4]:
PETPHOTOPATH='/Users/miaCDIPS/petphotos/'
TESTINPUTPATH = "/Users/miaCDIPS/Desktop/testpics/"

In [5]:
conn = sqlite3.connect('lookalike2.db')


In [6]:
pfdf = pd.read_pickle('pfdf_clean.pkl')

In [9]:
cursor = conn.cursor()

In [10]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[]


In [ ]:
cursor = conn.execute('select * from pf_vectors_0001_1')

In [ ]:
colnames = [description[0] for description in cursor.description]

# LET'S INPUT AN IMAGE HERE
Here, we have already trained our neural network, and input an image to obtain the model's prediction (which breed of dog), but more importantly, to extract the feature vector used to describe the image.

In [ ]:
input_image_filepath = TESTINPUTPATH + "mutt.jpg"
inputimg = iImage(filename = input_image_filepath, height = 600)
display(inputimg)

## Our re-trained neural network can predict which breed:

In [ ]:
topfivebreeds.top5graph(input_image_filepath)

## but even several categorical labels fail to capture the complexity of a dog's appearance.  Instead let's describe our dog with a 2048-dimensional feature _vector_...

## ...and compare against the other dogs in our image database.  
We measure the cosine similarity between our input and every other dog in our database, getting a similarity score for each.  Let's take a look at the 10 most similar.

In [ ]:
table_list = []
for i in range(1,31):
    table_list.append(['pf_vectors_'+'0'*(3-math.floor(math.log10(i)))+str(i)+'_1',
                       'pf_vectors_'+'0'*(3-math.floor(math.log10(i)))+str(i)+'_2'])
table_list_red = table_list[:3]

In [ ]:
table_list

In [ ]:
df = pd.merge(pd.read_sql("SELECT * FROM pf_vectors_0001_1",conn,index_col='index'),
              pd.read_sql("SELECT * FROM pf_vectors_0001_2",conn,index_col='index'),
              left_index=True,right_index=True)

In [ ]:
df.memory_usage(index=True,deep=True).sum()

In [ ]:
df.head()

In [ ]:
%%time
input_vector = extract_feature_vector.run_inference_on_image(input_image_filepath)
    


In [ ]:
%%time
similarities=pd.Series()
for tablepair in table_list_red:
    df = pd.merge(pd.read_sql("SELECT * FROM "+tablepair[0],conn,index_col='index'),
                  pd.read_sql("SELECT * FROM "+tablepair[1],conn,index_col='index'),
                  left_index=True,right_index=True)              
    similarities=similarities.append(pd.Series(np.squeeze(pairwise.cosine_similarity(df,input_vector.reshape(1,-1))), index=df.index))
    del df

In [ ]:
%%time
simpetsdf = maketop10df(similarities.sort_values(ascending=False,kind='heapsort').head(10))


In [ ]:
featdf = pd.read_sql(querystr,conn,index_col='index').transpose()

In [ ]:
featdf.memory_usage(deep=True)

In [ ]:
featdf.memory_usage(index=True,deep=True).sum()

In [ ]:
display(inputimg)
style = "<style> td{font-size: 16px;}</style>"
HTML(style+simpetsdf.to_html(escape=False,index=False).replace('\\n','<br>'))

In [ ]:
simpetsdf.to_html(formatters={'Picture': image_formatter}, escape=False,index=False).replace('\\n','<br>')

In [ ]:
top10list

In [ ]:
pfdf.loc['40540182']

In [ ]:
conn.close()

In [ ]:
import sqlite3
lapasql = sqlite3.connect("lookalike.db")

In [ ]:
sqlite3.version_info

In [ ]:
sqlite3.sqlite_version_info

In [ ]:
featuredf.to_sql(name="pf_image_vectors", con=lapasql, if_exists="append")

In [ ]:
pfdf2 = pd.read_pickle("pfdfFULL.pkl")

In [ ]:
pfdf2.media.iloc[0]

In [ ]:
for i in pfdf.index:
    pfdf.media.loc[i] = pfdf2.media.loc

In [ ]:
pfdf.media.iloc[0]

In [ ]:
pfdf['media2'] = pfdf2['media']

In [ ]:
pfdf = pfdf.drop(['media'], axis=1)

In [ ]:
pfdf = pfdf.rename(index=str, columns={'media2': "media"})

In [ ]:
pfdf.to_pickle("pfdf_clean.pkl")